# Pinit Recommendation Sandbox
Interactive notebook to experiment with each stage of the local Pinit recommendation pipeline without Supabase.


## Workflow Overview
1. Configure paths and helpers.
2. Load the canonical location inventory from CSVs.
3. Derive taxonomy + location tags (deterministic + reviews).
4. Build/synthesize user actions and compute taste profiles.
5. Generate recommendation scores + inspect outputs.
6. Persist artifacts for downstream experiments.


In [ ]:
from pathlib import Path
import json
import pandas as pd

from config import PipelineConfig, PipelinePaths, ReviewTagConfig
from recommendation.tag_taxonomy import tag_dataframe
from recommendation.static_tagging import load_locations, load_reviews, build_location_tags
from recommendation.user_profiles import ensure_user_actions, build_user_tag_affinities
from recommendation.recommendation import build_recommendations


### Configure paths + output folders


In [5]:
DATA_DIR = Path("../../data/raw")
CITY_NAME = "london"
OUTPUT_DIR = Path("../../output/pinit_notebook")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

paths = PipelinePaths(data_dir=DATA_DIR, city_name=CITY_NAME, output_dir=OUTPUT_DIR)
review_cfg = ReviewTagConfig(min_unique_authors=2, min_mentions=3)
config = PipelineConfig(paths=paths, review_tagging=review_cfg, top_k_per_user=25)
config


PipelineConfig(paths=PipelinePaths(data_dir=PosixPath('../../data/raw'), city_name='london', output_dir=PosixPath('../../output/pinit_notebook'), user_actions_csv=None, user_friends_csv=None, bubble_locations_csv=None), review_tagging=ReviewTagConfig(min_unique_authors=2, min_mentions=3, english_only=True, score_floor=20.0, score_cap=100.0), recommendation_weights=RecommendationWeights(taste=0.5, trend_app=0.15, hidden_gems=0.2, quality=0.15, friend=0.0, bubble=0.0), synthetic_users=True, top_k_per_user=25)

## 1. Load canonical location inventory


In [6]:
locations = load_locations(paths)
print(f"Loaded {len(locations):,} locations")
locations.head()


Loaded 1,608 locations


,place_id,name,types,rating,user_ratings_total,price_level,lat,lon,vicinity,business_status,...,price_bucket,log_reviews,popularity_score,expected_popularity,residual_popularity,quality_score,hidden_gem_score,hidden_gem_source,expected_rating,hype_residual
0,ChIJNSXdCgBT30cRJCJlCeqYZq4,Piccolo's Pizza,"establishment,food,point_of_interest,restaurant",NaN,0.0,NaN,51.283470,0.169840,"Riverhead, Sevenoaks",OPERATIONAL,...,unknown,0.000000,0.000000,4.028667,-4.028667,0.811178,0.734039,popularity_residual,NaN,NaN
1,ChIJB7xIORNS30cRSpYmpEaUvvM,Miller & Carter Sevenoaks,"establishment,food,point_of_interest,restaurant",4.5,1895.0,2.0,51.283125,0.170598,"Amherst Hill, Riverhead, Sevenoaks",OPERATIONAL,...,mid,7.547502,0.842022,5.480125,2.067377,0.875000,0.000000,popularity_residual,NaN,NaN
2,ChIJZa00u2xS30cRYBr8E5bIYdM,Sun On,"establishment,food,meal_takeaway,point_of_inte...",3.6,40.0,NaN,51.284885,0.169742,"27 London Rd, Riverhead, Sevenoaks",OPERATIONAL,...,unknown,3.713572,0.414297,4.329621,-0.616049,0.650000,0.112247,popularity_residual,NaN,NaN
3,ChIJTW54IChT30cR_lIlYS9uZZI,Linden Catering,"establishment,food,point_of_interest,restaurant",NaN,0.0,NaN,51.289080,0.168472,"Riverpoint house, London Rd, Sevenoaks",OPERATIONAL,...,unknown,0.000000,0.000000,2.186523,-2.186523,0.811178,0.398393,popularity_residual,NaN,NaN
4,ChIJ7_uwBeRT30cRt_w4ylhWfF4,Kanosh Lebanese Street Food,"establishment,food,meal_takeaway,point_of_inte...",5.0,4.0,NaN,51.289309,0.168325,"London Rd, Sevenoaks",CLOSED_TEMPORARILY,...,unknown,1.609438,0.179554,4.053022,-2.443584,1.000000,0.445231,popularity_residual,NaN,NaN


In [7]:
locations[['location_id','name','cuisine_primary','price_level','rating','user_ratings_total','is_open_late','is_open_early']].head(10)


,location_id,name,cuisine_primary,price_level,rating,user_ratings_total,is_open_late,is_open_early
0,1,Piccolo's Pizza,italian,NaN,NaN,0.0,False,False
1,2,Miller & Carter Sevenoaks,unknown,2.0,4.5,1895.0,True,False
2,3,Sun On,indian,NaN,3.6,40.0,False,False
3,4,Linden Catering,unknown,NaN,NaN,0.0,False,False
4,5,Kanosh Lebanese Street Food,middle_eastern,NaN,5.0,4.0,False,False
5,6,E K B GOURMET BURGER,american,NaN,3.0,3.0,True,False
6,7,M&S Food To Go,unknown,2.0,2.8,98.0,False,True
7,8,The Bell,pub,2.0,4.4,877.0,True,False
8,9,Trattoria Da Carlo,italian,NaN,4.5,69.0,False,False
9,10,Zen Garden Chinese Restaurant,chinese,2.0,4.2,260.0,False,False


In [8]:
place_to_location = locations.set_index('google_place_id')['location_id'].to_dict()

reviews = load_reviews(paths, place_to_location)
print(f"Loaded {len(reviews):,} reviews")

place_to_location

Loaded 7,240 reviews


{'ChIJNSXdCgBT30cRJCJlCeqYZq4': 1,
 'ChIJB7xIORNS30cRSpYmpEaUvvM': 2,
 'ChIJZa00u2xS30cRYBr8E5bIYdM': 3,
 'ChIJTW54IChT30cR_lIlYS9uZZI': 4,
 'ChIJ7_uwBeRT30cRt_w4ylhWfF4': 5,
 'ChIJHbX02-1T30cRDO1efNmH4nU': 6,
 'ChIJAcqe0zzZdUgR3pDe79_xkh0': 7,
 'ChIJ_bgO35PgdUgR3QVkvi-fwnc': 8,
 'ChIJMYLkfI7gdUgRFesJVMX5PPo': 9,
 'ChIJcz21cuXgdUgROfpWnFBuIoA': 10,
 'ChIJ1R6NmV__dUgROJgVsZpFaR4': 11,
 'ChIJj-4uTIxN30cRNFTyf69vF2c': 12,
 'ChIJCzhCNNJN30cR4kIUhhYJCvA': 13,
 'ChIJg55-TZnhdUgRB2wihE4l7lE': 14,
 'ChIJi1GLWQXhdUgRdZKBQ0UZ4oQ': 15,
 'ChIJ9Zh7jtdT30cRlLD0oyKtUTQ': 16,
 'ChIJxV42umdS30cRnImpcY0Qacc': 17,
 'ChIJ0ZC0oit2dkgRv9sDyGigQ1Y': 18,
 'ChIJy0c3bIXYdUgRojTzdhXE09Y': 19,
 'ChIJiTJbt8Wq2EcRXgu3uGInfTM': 20,
 'ChIJm2WXSsGq2EcRGlY-fs6mu7s': 21,
 'ChIJB1SV1rh1dkgR0uwwdRa90KA': 22,
 'ChIJl0L28AB1dkgRdwkc7DrdUjE': 23,
 'ChIJ2V0PyugKdkgRdgvkKB8LRo0': 24,
 'ChIJf7oNL-EKdkgRZScn5_R4pUg': 25,
 'ChIJS9H60PEKdkgRfXwMTP7hkTo': 26,
 'ChIJuSEJusoLdkgRKhVUhl5TOrg': 27,
 'ChIJabRCmzgKdkgRHP9oehOYowE': 28,
 

## 2. Build taxonomy + location tags


In [9]:
tags_df = tag_dataframe()
reviews_df = load_reviews(paths, locations.set_index('google_place_id')['location_id'].to_dict())
location_tags = build_location_tags(locations, reviews_df, config.review_tagging)
print(f"Tags defined: {len(tags_df)} | Tagged pairs: {len(location_tags):,}")


Tags defined: 47 | Tagged pairs: 6,465


In [10]:
tags_df
# write tags_df into a csv file
tags_df.to_csv(OUTPUT_DIR / "tags.csv", index=False)

In [11]:
location_tags_with_names = location_tags.merge(locations[["location_id", "name"]], on="location_id")
location_tags_with_names

,location_id,tag_text,score,source,metadata,tag_id,name
0,1,italian,92.000000,cuisine_detected,"{""field"": ""cuisine_primary""}",1,Piccolo's Pizza
1,1,restaurant,75.000000,google_types,"{""type"": ""restaurant""}",44,Piccolo's Pizza
2,2,restaurant,75.000000,google_types,"{""type"": ""restaurant""}",44,Miller & Carter Sevenoaks
3,2,open_late,70.000000,opening_hours,{},38,Miller & Carter Sevenoaks
4,2,sunday_open,65.000000,opening_hours,{},40,Miller & Carter Sevenoaks
...,...,...,...,...,...,...,...
6460,1595,family_friendly,47.465307,reviews,"{""mentions"": 2, ""unique_authors"": 2}",28,Rootz
6461,1598,family_friendly,47.465307,reviews,"{""mentions"": 2, ""unique_authors"": 2}",28,Sathi Restaurant Chorleywood
6462,1600,quiet,47.465307,reviews,"{""mentions"": 2, ""unique_authors"": 2}",24,The Bank Chorleywood
6463,1604,lively,47.465307,reviews,"{""mentions"": 2, ""unique_authors"": 2}",23,Nonno's Pizza (Chorleywood)


## 3. Build/simulate user actions + taste profiles


In [12]:
user_actions, synthetic = ensure_user_actions(paths, locations, location_tags, allow_synthetic=config.synthetic_users)
print(f"Loaded {len(user_actions)} user actions | synthetic={synthetic}")
user_actions


Loaded 36 user actions | synthetic=True


,user_id,place_id,action,created_at
0,demo_date_night,ChIJNSXdCgBT30cRJCJlCeqYZq4,like,2025-10-22T14:28:28.198186+00:00
1,demo_date_night,ChIJMYLkfI7gdUgRFesJVMX5PPo,detail_view,2025-11-10T14:28:28.199197+00:00
2,demo_date_night,ChIJj-4uTIxN30cRNFTyf69vF2c,like,2025-12-01T14:28:28.199236+00:00
3,demo_date_night,ChIJi1GLWQXhdUgRdZKBQ0UZ4oQ,detail_view,2025-12-11T14:28:28.199269+00:00
4,demo_date_night,ChIJy0c3bIXYdUgRojTzdhXE09Y,like,2025-10-16T14:28:28.199306+00:00
5,demo_date_night,ChIJDziemhAHdkgRPgRx1c7zC-8,save,2025-10-10T14:28:28.199340+00:00
6,demo_date_night,ChIJleBWrQULdkgRstOAcogkSFc,like,2025-11-04T14:28:28.199370+00:00
7,demo_date_night,ChIJ0X6uEBELdkgRlWlZh5uIRX8,detail_view,2025-11-16T14:28:28.199399+00:00
8,demo_date_night,ChIJ5ekJCS5xdkgRUbU1cErJ11g,like,2025-11-13T14:28:28.199428+00:00
9,demo_date_night,ChIJUyVgzv1zdkgR08_8R-kLGJ0,like,2025-10-06T14:28:28.199456+00:00


In [13]:
user_tags, user_history = build_user_tag_affinities(user_actions, location_tags, locations)
print(f"Computed {len(user_tags)} user-tag affinities across {user_tags['user_id'].nunique()} users")
user_tags.head(10)


Computed 39 user-tag affinities across 3 users


/Users/sriharshavitta/Projects/pinit-recommendations/src/pinit/user_profiles.py:156: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  normalized = agg.groupby("user_id", group_keys=False).apply(_normalize)


,user_id,tag_id,tag_text,score,metadata
0,demo_date_night,1,italian,100.000000,"{""raw_score"": 6.166226182463444}"
1,demo_date_night,44,restaurant,81.521739,"{""raw_score"": 5.026814822660416}"
2,demo_date_night,47,takeaway,79.523405,"{""raw_score"": 4.903592989468822}"
3,demo_date_night,40,sunday_open,66.165839,"{""raw_score"": 4.0799352855448}"
4,demo_date_night,38,open_late,57.568438,"{""raw_score"": 3.5498001049651977}"
5,demo_date_night,46,bar,8.309101,"{""raw_score"": 0.5123579809348315}"
6,demo_date_night,41,great_value,7.600534,"{""raw_score"": 0.46866614826631425}"
7,demo_date_night,39,open_early,6.650468,"{""raw_score"": 0.41008287973302493}"
8,demo_group_hang,44,restaurant,100.000000,"{""raw_score"": 5.183389863654495}"
9,demo_group_hang,40,sunday_open,83.138164,"{""raw_score"": 4.309375161226154}"


## 4. Generate recommendation scores


In [14]:
recs = build_recommendations(locations, user_tags, location_tags, user_history, user_actions, config)
print(f"Generated {len(recs)} rec rows")


Generated 75 rec rows


In [15]:
def show_user_recs(user_id, n=10):
    subset = recs[recs['user_id'] == user_id].merge(
        locations[['location_id','name','vicinity','cuisine_primary','rating','user_ratings_total']],
        on='location_id', how='left'
    ).head(n)
    return subset[['rank','score','name','cuisine_primary','rating','user_ratings_total','reason']]

unique_users = recs['user_id'].unique().tolist()
unique_users[:3], len(unique_users)


(['demo_date_night', 'demo_group_hang', 'demo_vegan'], 3)

In [16]:
# Example: inspect top recommendations for the first user
if len(recs):
    example_user = recs['user_id'].iloc[0]
show_user_recs(example_user, n=10)


,rank,score,name,cuisine_primary,rating,user_ratings_total,reason
0,1,2.041038,Nonno's Pizza (Chorleywood),italian,4.5,392.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."
1,2,2.038603,Domino's Pizza - Chigwell,italian,4.3,57.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."
2,3,2.024309,Just Pizza Rickmansworth,italian,4.0,1.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."
3,4,2.024218,Pizza Go Go,italian,4.1,96.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."
4,5,2.021643,Pizza King Kebab House,italian,3.9,494.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."
5,6,2.020989,Mascalzone,italian,4.3,1194.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."
6,7,2.019286,County Fried Chicken & pizza & Burgers,italian,4.0,102.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."
7,8,2.017219,Domino's Pizza - Brentwood,italian,4.0,96.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."
8,9,2.016788,Tops Pizza,italian,4.0,295.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."
9,10,2.014908,Farmhouse pizza Edgware,italian,3.9,330.0,"{""taste_tags"": [{""tag"": ""italian"", ""score"": 0...."


## 5. Persist notebook artifacts


In [17]:
# locations.to_csv(OUTPUT_DIR / 'locations.csv', index=False)
# location_tags.to_csv(OUTPUT_DIR / 'location_tags.csv', index=False)
# user_tags.to_csv(OUTPUT_DIR / 'user_tag_affinities.csv', index=False)
# recs.to_csv(OUTPUT_DIR / 'user_recommendations.csv', index=False)
# metadata = {
#     'city': CITY_NAME,
#     'n_locations': int(len(locations)),
#     'n_tags': int(len(tags_df)),
#     'n_location_tags': int(len(location_tags)),
#     'n_users': int(user_tags['user_id'].nunique()) if not user_tags.empty else 0,
#     'n_recommendations': int(len(recs)),
#     'synthetic_user_actions': bool(synthetic),
# }
# (OUTPUT_DIR / 'metadata.json').write_text(json.dumps(metadata, indent=2))
# metadata


## Processing location_tags

In [18]:
# Choose Tags that are not in supabase
new_tags = tags_df[tags_df['supabase_id'].isnull()]   
new_tags.drop(columns=['tag_id', 'supabase_id', 'slug'], inplace=True)
new_tags = new_tags[['text', 'prompt_description', 'tag_type', 'color']]
new_tags.rename(columns={'color': 'colour'}, inplace=True)
new_tags 

/var/folders/w5/q3g22k8d1_72830gw6496kz00000gn/T/ipykernel_19293/3551327770.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_tags.drop(columns=['tag_id', 'supabase_id', 'slug'], inplace=True)


,text,prompt_description,tag_type,colour
9,british,Modern British kitchens and proper roasts.,CUISINE,#F94144
10,pub,True pub fare with pints and Sunday sessions.,CUISINE,#F94144
11,bakery,"Bakeries, patisseries and pastry labs.",CUISINE,#F94144
13,seafood,"Raw bars, shellfish shacks and seafood grills.",CUISINE,#F94144
15,vegan_vegetarian,Veggie-first kitchens and plant-based menus.,CUISINE,#F94144
16,vegetarian_friendly,Menus with strong vegetarian sections.,DIETARY,#90BE6D
17,vegan_friendly,Vegan-friendly options beyond token salads.,DIETARY,#90BE6D
18,halal_friendly,Halal-friendly kitchens.,DIETARY,#90BE6D
19,gluten_free_options,Staff that knows their gluten-free swaps.,DIETARY,#90BE6D
26,formal,White tablecloths or tasting menus.,VIBE,#577590


In [19]:
new_tags.to_csv(OUTPUT_DIR / "new_tags.csv", index=False)

## Supabase setup 

In [20]:
import supabase_config as sc
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Supabase service
service = sc.get_supabase_service()

# Test connection
print("Testing connection...")
if service.test_connection():
    print("✓ Connected to Supabase")
else:
    print("✗ Connection failed")

# Get all tags from tag_taxonomy
tags_from_db = service.get_all_tags()
print(f"✓ Loaded {len(tags_from_db)} tags from tag_taxonomy")

# Create mapping from tag text to UUID
tag_text_to_uuid = {tag['text']: tag['tag_id'] for tag in tags_from_db}
print(f"✓ Created tag mapping with {len(tag_text_to_uuid)} entries")

# Show sample
if tags_from_db:
    sample = tags_from_db[0]
    print(f"\nSample tag: {sample['text']} -> {sample['tag_id']}")


Testing connection...
✓ Connected to Supabase
✓ Loaded 104 tags from tag_taxonomy
✓ Created tag mapping with 104 entries

Sample tag: casual -> 7f789b3e-ae55-4ac1-83fb-5ba3c785d793


## Format location_tags for Supabase upload

In [22]:
# Load location_tags from the pipeline (already created above)
print(f"Original location_tags shape: {location_tags.shape}")
print(f"Columns: {location_tags.columns.tolist()}")

# Inspect first few rows to understand structure
print("\nFirst few rows:")
print(location_tags.head())

# Create a copy for Supabase formatting
location_tags_supabase = location_tags.copy()

# Identify the tag column - it might be 'tag_text', 'tag', or similar
tag_column = None
for col in ['tag_text', 'tag', 'text']:
    if col in location_tags_supabase.columns:
        tag_column = col
        break

if tag_column is None:
    raise ValueError(f"Could not find tag column in: {location_tags_supabase.columns.tolist()}")

print(f"\nUsing '{tag_column}' as the tag identifier column")

# Map the tag text to UUID from Supabase
location_tags_supabase['tag_id'] = location_tags_supabase[tag_column].map(tag_text_to_uuid)

# Check for unmapped tags
unmapped = location_tags_supabase[location_tags_supabase['tag_id'].isnull()]
if len(unmapped) > 0:
    print(f"\n⚠ Warning: {len(unmapped)} location_tags have unmapped tags:")
    print(f"Unmapped tags: {sorted(unmapped[tag_column].unique()[:10])}")
else:
    print("✓ All tags successfully mapped to UUIDs")

# Remove rows with unmapped tags
location_tags_supabase = location_tags_supabase[location_tags_supabase['tag_id'].notna()]

# Format according to Supabase schema:
# location_id, tag_id (uuid), score, source, metadata

# Rename confidence column to score if it exists
if 'confidence' in location_tags_supabase.columns:
    location_tags_supabase = location_tags_supabase.rename(columns={'confidence': 'score'})
elif 'score' not in location_tags_supabase.columns:
    # If no score column, set default
    location_tags_supabase['score'] = 1.0

# Add source column (indicates whether tag came from rules or reviews)
if 'source' not in location_tags_supabase.columns:
    location_tags_supabase['source'] = 'pipeline'

# Get tag_type from tags_df if available
tag_type_map = tags_df.set_index('text')['tag_type'].to_dict() if 'text' in tags_df.columns else {}

# Create metadata as JSON (can include original tag text for reference)
location_tags_supabase['metadata'] = location_tags_supabase.apply(
    lambda row: {
        'original_tag': row[tag_column], 
        'tag_type': tag_type_map.get(row[tag_column], 'unknown')
    },
    axis=1
)

# Select final columns for Supabase
final_columns = ['location_id', 'tag_id', 'score', 'source', 'metadata']
location_tags_final = location_tags_supabase[final_columns].copy()

# Convert metadata dict to JSON string for CSV export
location_tags_final['metadata'] = location_tags_final['metadata'].apply(json.dumps)

print(f"\nFinal location_tags shape: {location_tags_final.shape}")
print(f"Columns: {location_tags_final.columns.tolist()}")
print(f"\nSample rows:")
location_tags_final.head()

Original location_tags shape: (6465, 6)
Columns: ['location_id', 'tag_text', 'score', 'source', 'metadata', 'tag_id']

First few rows:
   location_id     tag_text  score            source  \
0            1      italian   92.0  cuisine_detected   
1            1   restaurant   75.0      google_types   
2            2   restaurant   75.0      google_types   
3            2    open_late   70.0     opening_hours   
4            2  sunday_open   65.0     opening_hours   

                       metadata  tag_id  
0  {"field": "cuisine_primary"}       1  
1        {"type": "restaurant"}      44  
2        {"type": "restaurant"}      44  
3                            {}      38  
4                            {}      40  

Using 'tag_text' as the tag identifier column

⚠ Warning: 103 location_tags have unmapped tags:
Unmapped tags: ['family_friendly', 'quick_bite']

Final location_tags shape: (6362, 5)
Columns: ['location_id', 'tag_id', 'score', 'source', 'metadata']

Sample rows:


,location_id,tag_id,score,source,metadata
0,1,d0ae158d-cedd-4f29-9c03-6ebfe2c0a76c,92.0,cuisine_detected,"{""original_tag"": ""italian"", ""tag_type"": ""CUISI..."
1,1,51eec92e-5a7d-4b2f-a96c-dc364b1b253a,75.0,google_types,"{""original_tag"": ""restaurant"", ""tag_type"": ""CA..."
2,2,51eec92e-5a7d-4b2f-a96c-dc364b1b253a,75.0,google_types,"{""original_tag"": ""restaurant"", ""tag_type"": ""CA..."
3,2,eaa9d86e-1f30-4198-b4c8-bed603ce5d3a,70.0,opening_hours,"{""original_tag"": ""open_late"", ""tag_type"": ""SCH..."
4,2,ec1bf490-469c-410c-97e7-6034a75ce9e2,65.0,opening_hours,"{""original_tag"": ""sunday_open"", ""tag_type"": ""S..."


In [23]:
# Save to CSV for Supabase upload
output_path = OUTPUT_DIR / "location_tags.csv"
location_tags_final.to_csv(output_path, index=False)
print(f"✓ Saved location_tags to: {output_path}")
print(f"  Total records: {len(location_tags_final):,}")
print(f"  Unique locations: {location_tags_final['location_id'].nunique():,}")
print(f"  Unique tags: {location_tags_final['tag_id'].nunique():,}")

✓ Saved location_tags to: ../../output/pinit_notebook/location_tags.csv
  Total records: 6,362
  Unique locations: 1,608
  Unique tags: 31


## Optional: Upload location_tags directly to Supabase

In [25]:
# Uncomment to upload directly to Supabase (requires service key)
# WARNING: This will insert all location_tags into Supabase

UPLOAD_TO_SUPABASE = True  # Set to True to enable upload

if UPLOAD_TO_SUPABASE:
    # Convert to dict format for Supabase (without JSON string for metadata)
    location_tags_upload = location_tags_supabase[final_columns].copy()
    
    # Keep metadata as dict for proper JSONB storage
    records = location_tags_upload.to_dict('records')
    
    print(f"Uploading {len(records):,} location_tags to Supabase...")
    
    # Upload in batches (Supabase has limits on batch size)
    batch_size = 1000
    total_inserted = 0
    
    for i in range(0, len(records), batch_size):
        batch = records[i:i + batch_size]
        try:
            result = service.client.table("location_tags").insert(batch).execute()
            total_inserted += len(result.data)
            print(f"  Batch {i//batch_size + 1}: Inserted {len(result.data)} records")
        except Exception as e:
            print(f"  Batch {i//batch_size + 1}: Error - {e}")
            break
    
    print(f"\n✓ Total inserted: {total_inserted:,} location_tags")
else:
    print("Upload disabled. Set UPLOAD_TO_SUPABASE = True to upload.")
    print(f"CSV file ready for manual import: {output_path}")

Uploading 6,362 location_tags to Supabase...
  Batch 1: Inserted 1000 records
  Batch 2: Inserted 1000 records
  Batch 3: Inserted 1000 records
  Batch 4: Inserted 1000 records
  Batch 5: Inserted 1000 records
  Batch 6: Inserted 1000 records
  Batch 7: Inserted 362 records

✓ Total inserted: 6,362 location_tags
